In [1]:
import os
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

In [40]:
def load_correct_study(patient_path):
    for root, dirs, files in os.walk(patient_path):
        #print(root, dirs, files)
        dcm_files = [f for f in files if f.endswith(".dcm")]
        if len(dcm_files) == 60:
            return os.path.join(root)
    return None

In [58]:
count_no_slices = []

In [52]:
def load_patient_volume(patient_folder):
    """
    Load the 60-slice DICOM volume for a patient.
    Returns:
        volume_np: (Z,H,W) float32 numpy array
    """
    # Step 1: find the folder with exactly 60 .dcm files
    subfolders = [os.path.join(patient_folder, f) for f in os.listdir(patient_folder) 
                  if os.path.isdir(os.path.join(patient_folder, f))]
    
    study_folder = load_correct_study(patient_folder)
    if study_folder is None:
        #print(f"No 60-slice folder found in {patient_folder}")
        return None
    
    # Step 2: read slices
    dcm_files = sorted([os.path.join(study_folder,f) for f in os.listdir(study_folder) if f.lower().endswith('.dcm')])
    slices = []
    for f in dcm_files:
        img = sitk.ReadImage(f)
        arr = sitk.GetArrayFromImage(img)[0]  # (1,H,W) -> (H,W)
        slices.append(arr.astype(np.float32))
    
    # Step 3: stack into volume
    volume_np = np.stack(slices, axis=0)  # (Z,H,W)
    
    return volume_np

In [53]:
def generate_consecutive_triplets(volume,j):
    """
    Generate overlapping triplets: (slice[i], slice[i+2]) -> slice[i+1]
    Returns lists of numpy arrays: pre_slices, post_slices, middle_slices
    """
    pre_slices = []
    post_slices = []
    middle_slices = []
    
    for i in range(volume.shape[0]-2):
        pre_slices.append(volume[i])
        post_slices.append(volume[i+2])
        middle_slices.append(volume[i+1])
        
    patients = [j for _ in range(len(pre_slices))]
    
    return pre_slices, post_slices, middle_slices, patients

In [54]:
parent_directory = os.path.dirname(os.getcwd())
parent_directory.split('/')[-1]

'Multi-Image-Super-Resolution'

In [55]:
def generate_progressive_triplets(volume,i):
    """
    volume: (Z,H,W)
    Returns: lists of pre_slices, post_slices, middle_slices
    """
    triplets = []
    
    def recursive_triplets(indices):
        if len(indices) < 2:
            return
        start = indices[0]
        end = indices[1]
        if(start>end or end-start<=2):
            return
        mid = (start + end) // 2
        if (mid != start and mid != end):
            # add triplet: start & end -> mid
            triplets.append((volume[start], volume[end], volume[mid]))
            # recurse on left and right halves
            recursive_triplets([start, mid])
            recursive_triplets([mid, end])
    
    recursive_triplets([0, volume.shape[0]-1])
    
    # Unpack triplets
    pre_slices = [t[0] for t in triplets]
    post_slices = [t[1] for t in triplets]
    middle_slices = [t[2] for t in triplets]
    patients = [i for _ in range(len(pre_slices))]
    
    return pre_slices, post_slices, middle_slices, patients

In [ ]:
BASE_DIR = os.path.join(parent_directory, 'data','manifest-1694710246744','Prostate-MRI-US-Biopsy')
patient_folders = sorted([f for f in os.listdir(BASE_DIR) if f.startswith("Prostate-MRI-US-Biopsy-")])

all_pre, all_post, all_middle = [], [], []
patients = []

for i,pid in enumerate(patient_folders):
    patient_path = os.path.join(BASE_DIR, pid)
    volume = load_patient_volume(patient_path)
    if volume is None:
        count_no_slices.append(pid)
        continue
    
    pre, post, middle, patient_number = generate_consecutive_triplets(volume,i)
    all_pre.extend(pre)
    all_post.extend(post)
    all_middle.extend(middle)
    patients.extend(patient_number)
    
    pre, post, middle, patient_number = generate_progressive_triplets(volume,i)
    all_pre.extend(pre)
    all_post.extend(post)
    all_middle.extend(middle)
    patients.extend(patient_number)

In [ ]:
len(count_no_slices)

In [ ]:
count_no_slices